So what am I looking to do?
Train an epoch:

We should have already defined the epoch in terms of the training data.
We 

Train several epochs
Test 
monitor during test
graph results
information theory

When we run the network we wnat

Make sure to delete data after the run

So I think run standalone mode for the training, and runtime for testing (apparently allows more flexibility?)
I can't remember.

I am using Brian2 to run a simulation of a spiking neural network.
It is a visnet model with n_layers, and in each layer an excitatory and inhibitory layer.
Each neuron group is named according to <type>_layer_<layer_number> [type here is "excitatory"] (indexed from 1). The input layer is called input_layer_0, treat this as a member of the excitatory neurons so if we ask for layer 0 record from this one.

I want you to make a function which takes as input a Network object, a list of layers, and the type of monitor I want made for these layers (spike, voltage, pop_avg_spike, time_avg_spike).
It will have to find the corresponding objects and then pass that object to a function which makes those Monitors in brian2.
Right now create a gating function which gets the right object and passes it to the right function and with the Network as a parameter, checking that the layer list is within bounds of n_layers and raising errors if object cant be found.

In [ ]:
from brian2 import *
def run_training_epoch(network, stimulus_length, no_stimuli):
    network.run(stimulus_length * no_stimuli)

def run_training(network, stimulus_length, no_stimuli, no_epochs):
    for i in range(no_epochs):
        run_training_epoch(network, stimulus_length, no_stimuli)

def run_testing_epoch(network, stimulus_length, no_stimuli):
    network.run(stimulus_length * no_stimuli)

In [4]:
import sys
import os

# Add the parent directory of the current working directory
sys.path.append(os.path.dirname(os.getcwd()))

from brian2 import *
from input import *
from network import *


In [ ]:
from brian2 import Network, SpikeMonitor, StateMonitor, PopulationRateMonitor

#1. Additional Default Parameters: The use of a lambda function for the 'voltage' monitor allows for the inclusion of additional default parameters that are not part of the default constructor call for the other types of monitors. In the specific case of the 'voltage' monitor, the lambda function ensures that the record=True parameter is always included when the StateMonitor is instantiated:
def create_monitor(neuron_group, monitor_type, **kwargs):
    """
    Create and return the specified type of monitor for a given neuron group.
    
    Parameters:
    neuron_group (NeuronGroup): The neuron group to monitor.
    monitor_type (str): The type of monitor to create.
    **kwargs: Arbitrary keyword arguments for monitor configuration.
    
    Returns:
    brian2 monitor: The created monitor object.
    """
    monitor_factory = {
        'spike': SpikeMonitor,
        'voltage': lambda group, **kw: StateMonitor(group, 'v', record=True, **kw),
        'pop_avg_spike': PopulationRateMonitor,
        'time_avg_spike': PopulationRateMonitor
    }
    
    if monitor_type in monitor_factory:
        return monitor_factory[monitor_type](neuron_group, **kwargs)
    else:
        raise ValueError(f"Unsupported monitor type: {monitor_type}")

def setup_monitors(network, layers, monitor_type, n_layers, **kwargs):
    """
    Set up and return monitors for specified layers of a network.
    
    Parameters:
    network (Network): The Brian2 network instance.
    layers (list of int): Layer indices to monitor.
    monitor_type (str): Type of monitor to create.
    n_layers (int): Total number of layers in the network, excluding the input layer.
    **kwargs: Arbitrary keyword arguments for monitor configuration.
    
    Returns:
    list: List of created monitors.
    
    Raises:
    ValueError: If a layer index is out of bounds or neuron group is not found.
    """
    # Validate layer indices
    if not all(0 <= layer <= n_layers for layer in layers):
        raise ValueError("One or more layer indices are out of bounds.")

    # Map of neuron groups by their names
    group_map = {obj.name: obj for obj in network.objects if hasattr(obj, 'name')}

    monitors = []
    for layer in layers:
        layer_name = f"excitatory_layer_{layer}" if layer != 0 else "input_layer_0"
        
        if layer_name not in group_map:
            raise ValueError(f"Neuron group for layer {layer_name} not found.")
        
        try:
            monitor = create_monitor(group_map[layer_name], monitor_type, **kwargs)
            network.add(monitor)
            monitors.append(monitor)
        except Exception as e:
            raise ValueError(f"Error creating monitor for {layer_name}: {str(e)}")
    
    return monitors

# Example usage:
# net = Network(...)
# try:
#     monitors = setup_monitors(net, [0, 1, 2], 'voltage', n_layers=4, record=True)
# except ValueError as e:
#     print(e)

In [ ]:
G = NeuronGroup(...)
M = PopulationRateMonitor(G)
run(...)
plot(M.t/ms, M.rate/Hz)

In [ ]:
group = NeuronGroup(..., 'dv/dt = ... : volt', ...)

# Dummy group to store the average membrane potential at every time step
vm_container = NeuronGroup(1, 'average_vm : volt')

# Synapses averaging the membrane potential of all neurons in group
vm_averager = Synapses(group, vm_container, 'average_vm_post = v_pre/N_pre : volt (summed)')
vm_averager.connect()

# Monitor recording the average membrane potential
vm_monitor = StateMonitor(vm_container, 'average_vm', record=True)